In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import logging
# sys.path

from ai_schema.eval import eval_text_span_classify
from config_ai.backend import set_tf_config
from config_ai.utils import *
from config_ai.models.mlm import TFMLMModel, get_mlm_output, eval_mlm
from config_ai.data_utils import *
from config_ai.models import load_model
from config_ai.experiments import get_model_config
from config_ai.schema import MaskedLanguageModelExample

logging.basicConfig(level=logging.DEBUG, format="%(asctime)s [%(levelname)s][%(filename)s:%(lineno)d]:%(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S')
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


set_tf_config()

2021-10-27 11:06:57 [INFO][backend.py:57]:setting tensorflow config...
2021-10-27 11:06:57.408864: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-10-27 11:06:57 [INFO][backend.py:61]:current devices:
2021-10-27 11:06:57.438476: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
coreClock: 1.531GHz coreCount: 28 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 447.48GiB/s
2021-10-27 11:06:57.439367: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-10-27 11:06:57.445364: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-10-27 11:06:57.448747: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-1

In [3]:
import numpy as np
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer


In [13]:
pretrain_name = "chinese_roberta_wwm_ext_L-12_H-768_A-12"
pretrain_name = "chinese_rbt4_L-4_H-768_A-12"
pretrain_name = "chinese_L-12_H-768_A-12"


config_path = f'/nfs/pony/chenhao/pretrain/{pretrain_name}/config.json'
checkpoint_path = f'/nfs/pony/chenhao/pretrain/{pretrain_name}/model.ckpt'
dict_path = f'/nfs/pony/chenhao/pretrain/{pretrain_name}/vocab.txt'

In [14]:
tokenizer = Tokenizer(dict_path, do_lower_case=True)  # 建立分词器
model = build_transformer_model(
    config_path=config_path, checkpoint_path=checkpoint_path, with_mlm=True
)  # 建立模型，加载权重
model

In [15]:
def to_array(*args):
    """批量转numpy的array
    """
    results = [np.array(a) for a in args]
    if len(args) == 1:
        return results[0]
    else:
        return results

In [16]:
token_ids, segment_ids = tokenizer.encode(u'科学技术是第一生产力')

# mask掉“技术”
token_ids[3] = token_ids[4] = tokenizer._token_mask_id
token_ids, segment_ids = to_array([token_ids], [segment_ids])
# token_ids, segment_ids = [token_ids], [segment_ids]
token_ids, segment_ids 

(array([[ 101, 4906, 2110,  103,  103, 3221, 5018,  671, 4495,  772, 1213,
          102]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]))

In [17]:
[token_ids, segment_ids]
probas = model.predict([token_ids, segment_ids])[0]
probas
probas.shape

[array([[ 101, 4906, 2110,  103,  103, 3221, 5018,  671, 4495,  772, 1213,
          102]]),
 array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])]

array([[2.5784804e-08, 2.8556206e-08, 2.7555107e-08, ..., 8.9684562e-08,
        6.4718044e-08, 6.2687171e-08],
       [3.0661202e-08, 3.2667170e-08, 3.0340075e-08, ..., 1.0172227e-07,
        9.5783122e-08, 1.7476081e-07],
       [9.5591688e-18, 1.1546758e-17, 6.7759583e-18, ..., 5.8764678e-17,
        6.0488456e-17, 2.7724075e-17],
       ...,
       [2.0150147e-22, 5.1080855e-23, 1.2026558e-22, ..., 1.7970673e-20,
        5.1858772e-22, 2.8766597e-21],
       [1.5838940e-18, 1.3842768e-18, 1.6169397e-18, ..., 9.2541241e-17,
        1.1762511e-18, 2.2351850e-16],
       [2.7384844e-10, 3.0633091e-10, 1.7498809e-10, ..., 4.0032662e-09,
        1.0272554e-09, 1.3808950e-09]], dtype=float32)

(12, 21128)

In [18]:
tmp = np.argmax(probas, axis=1)
tmp
rs = tokenizer.decode(tmp)
rs
rs[3:5]
# print(tokenizer.decode(probas[3:5].argmax(axis=1)))

array([8024,  511, 2110, 2825, 3318, 3221, 5018,  671, 4495,  772, 1213,
       4906])

'，。学技术是第一生产力科'

'技术'